In [4]:
# Amazon Dell G15 5530 Laptop Scraper

import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

# MongoDB setup
client = MongoClient("mongodb://localhost:27017/")
db = client["buygenix"]
collection = db["dell_g15_5530"]

# Target product URL (Amazon product page)
url = "https://www.amazon.in/dp/B0D83B2XKP"

# Headers to avoid bot detection
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5"
}

# Send request
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Initialize variables with default values to prevent NameError
title = "NA"
price = None
rating = "NA"
description = "NA"

# --- Scrape data using robust try-except blocks ---
try:
    title_element = soup.find("span", {"id": "productTitle"})
    if title_element:
        title = title_element.get_text(strip=True)
except Exception as e:
    print(f"Error scraping title: {e}")

try:
    # First, try finding the price from the 'a-offscreen' class (most reliable)
    price_element = soup.find("span", {"class": "a-offscreen"})
    if price_element:
        price_text = price_element.get_text(strip=True)
    else:
        # If a-offscreen is not found, try the a-price-whole class
        price_element = soup.find("span", {"class": "a-price-whole"})
        if price_element:
            price_text = price_element.get_text(strip=True)

    if price_text:
        cleaned_price_text = price_text.replace("₹", "").replace(",", "").strip()
        price = float(cleaned_price_text)
except Exception as e:
    print(f"Error scraping price: {e}")

try:
    rating_element = soup.find("span", {"class": "a-icon-alt"})
    if rating_element:
        rating = rating_element.get_text(strip=True)
except Exception as e:
    print(f"Error scraping rating: {e}")

try:
    description_element = soup.find("div", {"id": "feature-bullets"})
    if description_element:
        description = description_element.get_text(strip=True)
except Exception as e:
    print(f"Error scraping description: {e}")

# --- Store in MongoDB ---
collection.insert_one({
    "title": title,
    "price": price,
    "rating": rating,
    "description": description,
    "url": url
})

print("Data stored in MongoDB")

Data stored in MongoDB
